In [1]:
import torch
from torch import nn, optim
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

file_path_image_data = "task2b/single_label_image_dataset/image_data_dim60.txt"
file_path_image_labels = "task2b/single_label_image_dataset/image_data_labels.txt"

image_data = np.loadtxt(file_path_image_data)
labels = np.loadtxt(file_path_image_labels)

group_data = {1:0,2:1,3:2,5:3,6:4}

NUM_CLASSES = 5
MAX_EPOCHS = 50
INPUT_DIM  = image_data.shape[1]
image_data = image_data[np.in1d(labels, list(group_data.keys()))]
labels = labels[np.in1d(labels, list(group_data.keys()))]

print(image_data.shape)
print(labels.shape)

labels = np.vectorize(group_data.get)(labels)


(1598, 60)
(1598,)


In [12]:
class MLFFNN(nn.Module):
    """
    Class of Multi Layer Feed Forward Neural Network (MLFFNN)
    """
    def __init__(self, hidden_dim) :
        super(MLFFNN, self).__init__()
        torch.manual_seed(3)
        # adding linear and non-linear hidden layers
        self.mlffnn = nn.Sequential(nn.Linear(INPUT_DIM, hidden_dim),
                                       nn.Tanh(),
                                       nn.Linear(hidden_dim, hidden_dim),
                                       nn.Tanh(),
                                       nn.Linear(hidden_dim, NUM_CLASSES))
        
    def forward(self, X):
        y = self.mlffnn(X)
        return y

hidden_dim = 48
BATCH_SIZE = 32
# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(image_data,labels, train_size=0.7)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.67)

X_train, y_train, X_valid, X_test, y_valid, y_test = map(torch.tensor, [X_train, y_train, X_valid, X_test, y_valid, y_test])

model_delta = MLFFNN(hidden_dim).to(device)
model_ada_delta = MLFFNN(hidden_dim).to(device)
model_adam = MLFFNN(hidden_dim).to(device)


In [13]:
accuracy_metrics = {
    'train': [],
    "val": []
}
loss_metrics = {
    'train': [],
    "val": []
}

In [14]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc

In [15]:
train_dataloader = DataLoader(TensorDataset(X_train.unsqueeze(1), y_train.unsqueeze(1)), batch_size=BATCH_SIZE,
                              pin_memory=True, shuffle=True)
valid_dataloader = DataLoader(TensorDataset(X_valid.unsqueeze(1), y_valid.unsqueeze(1)), batch_size=BATCH_SIZE,
                            pin_memory=True, shuffle=True)
test_dataloader = DataLoader(TensorDataset(X_test.unsqueeze(1), y_test.unsqueeze(1)), batch_size=BATCH_SIZE,
                            pin_memory=True, shuffle=True)


In [16]:

lrs = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
criterion = nn.CrossEntropyLoss()

def train_model(optimizer,model):
    for lr in lrs:
        print(lr)
        optimizer_delta = optim.SGD(model_delta.parameters(),momentum=0, lr=lr)
        optimizer_adaptive_delta = optim.SGD(model_ada_delta.parameters(), lr=lr)
        optimizer_adam  = optim.Adam(model_adam.parameters(), lr=lr)
        model_delta.train()
        loss_metrics = {
        'train': [],
        "val": []
        }
        
        for epoch in range(MAX_EPOCHS):
            train_loss = []
            train_acc = []
            for X_train, y_train in train_dataloader:
                X_train = X_train.type(torch.float32).to(device)
                y_train = y_train.type(torch.long).to(device)

                optimizer_delta.zero_grad()

                score = model_delta(X_train)
                score = score.squeeze()
                y_train = y_train.squeeze(1)
                
                loss = criterion(input=score, target=y_train)
                loss.backward()
                train_loss.append(loss.item())
                optimizer_delta.step()
        
            # Validation
            validation_loss = []
            validation_acc = []
            with torch.no_grad():
                for X_valid, y_valid in valid_dataloader:
                    model_delta.eval()
                    X_valid = X_valid.type(torch.float32).to(device)
                    y_valid = y_valid.type(torch.long).to(device)
                    
                    score = model_delta(X_valid)
                    score = score.squeeze()
                    
                    y_valid = y_valid.squeeze(1)
                    loss = criterion(input=score, target=y_valid)
                    validation_loss.append(loss.item())

            
            loss_metrics["train"].append(np.average(train_loss))
            loss_metrics["val"].append(np.average(validation_loss))
        
        
        print(np.average(loss_metrics["train"]))
        print(np.average(loss_metrics["val"]))


        


1e-06
1.6496777132579257
1.6619510550498964
1e-05
1.6445704381125312
1.6559455261230465
0.0001
1.6152365400450572
1.6228973712921144
0.001
1.5394811022622248
1.5386823434829713
0.01
0.997397154399327
1.0117304501533508
